# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [22]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [23]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,vilyuchinsk,52.9306,158.4028,8.85,93,100,6.65,RU,1696277244
1,1,adamstown,-25.0660,-130.1015,19.85,76,72,11.07,PN,1696277245
2,2,egilsstadir,65.2653,-14.3948,4.56,99,100,2.77,IS,1696277314
3,3,grytviken,-54.2811,-36.5092,2.64,97,100,5.84,GS,1696277407
4,4,albany,42.6001,-73.9662,25.51,50,44,0.89,US,1696276611


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [24]:
%%capture --no-display

# Configure the map plot
map_city=city_data_df.hvplot.points("Lng","Lat",geo=True,tiles="OSM",color="City",alpha=0.5,size="Humidity")

# Display the map
map_city

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [25]:
# Narrow down cities that fit criteria and drop any results with null values
# A max temperature lower than 27 degrees but higher than 21
# Wind speed less than 4.5 m/s
# Zero cloudiness

city_filter_df=city_data_df.loc[(city_data_df["Max Temp"]< 27)
                               & (city_data_df["Max Temp"] > 21)
                               & (city_data_df["Wind Speed"] < 4.5)
                               & (city_data_df["Cloudiness"] == 0)]
# Drop any rows with null values
city_filter_df.dropna()

# Display sample data
city_filter_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
19,19,juan de ayolas,-27.4000,-56.9000,26.34,40,0,2.88,PY,1696277408
95,95,rio grande,-32.0350,-52.0986,21.06,73,0,3.71,BR,1696277200
114,114,teguise,29.0605,-13.5640,24.12,73,0,1.54,ES,1696277293
122,122,arar,30.9753,41.0381,26.01,27,0,2.57,SA,1696277438
142,142,laguna,38.4210,-121.4238,26.27,41,0,3.60,US,1696277175
242,242,keene,42.9337,-72.2781,24.68,49,0,2.06,US,1696277447
250,250,lachhmangarh sikar,27.8167,75.0333,26.49,22,0,2.56,IN,1696277447
259,259,rawalpindi,33.6007,73.0679,21.80,68,0,2.46,PK,1696277402
283,283,javea,38.7833,0.1667,26.03,71,0,3.58,ES,1696277450
294,294,roma,43.2128,-75.4557,25.84,66,0,2.57,US,1696277164


### Step 3: Create a new DataFrame called `hotel_df`.

In [26]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = city_filter_df[["City", "Country","Lat","Lng","Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
19,juan de ayolas,PY,-27.4000,-56.9000,40,
95,rio grande,BR,-32.0350,-52.0986,73,
114,teguise,ES,29.0605,-13.5640,73,
122,arar,SA,30.9753,41.0381,27,
142,laguna,US,38.4210,-121.4238,41,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [27]:
# Set parameters to search for a hotel
radius = 10000
base_params = {"categories":"accommodation.hotel","apiKey":geoapify_key}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    longitude = row["Lng"]
    latitude = row["Lat"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    filters = f"circle:{longitude},{latitude},{radius}"
    bias = f"proximity:{longitude},{latitude}"
    
    params = dict(base_params, filter=filters,bias=bias)
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url,params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
juan de ayolas - nearest hotel: Hotel Nacional de Turismo Ayolas
rio grande - nearest hotel: Hotel Vila Moura Executivo
teguise - nearest hotel: Finca Las Laderas
arar - nearest hotel: دار أوتيل Dar Otel
laguna - nearest hotel: Holiday Inn Express & Suites
keene - nearest hotel: Fairfield Inn & Suites by Marriott Keene Downtown
lachhmangarh sikar - nearest hotel: Soni resort
rawalpindi - nearest hotel: Gulf Hotel
javea - nearest hotel: Hotel Sol de Jávea
roma - nearest hotel: Wingate by Wyndham Rome
barakani - nearest hotel: Al Amal
saint-pierre - nearest hotel: Tropic Hotel
crucecita - nearest hotel: No hotel found
el hamma - nearest hotel: No hotel found
pisco - nearest hotel: La Portada
de aar - nearest hotel: No hotel found
bhinmal - nearest hotel: No hotel found
chato - nearest hotel: RA Crest Hotel


,City,Country,Lat,Lng,Humidity,Hotel Name
19,juan de ayolas,PY,-27.4000,-56.9000,40,Hotel Nacional de Turismo Ayolas
95,rio grande,BR,-32.0350,-52.0986,73,Hotel Vila Moura Executivo
114,teguise,ES,29.0605,-13.5640,73,Finca Las Laderas
122,arar,SA,30.9753,41.0381,27,دار أوتيل Dar Otel
142,laguna,US,38.4210,-121.4238,41,Holiday Inn Express & Suites
242,keene,US,42.9337,-72.2781,49,Fairfield Inn & Suites by Marriott Keene Downtown
250,lachhmangarh sikar,IN,27.8167,75.0333,22,Soni resort
259,rawalpindi,PK,33.6007,73.0679,68,Gulf Hotel
283,javea,ES,38.7833,0.1667,71,Hotel Sol de Jávea
294,roma,US,43.2128,-75.4557,66,Wingate by Wyndham Rome


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [28]:
%%capture --no-display

# Configure the map plot
hotel_map = hotel_df.hvplot.points("Lng","Lat",geo=True,color="City",alpha=0.5,
                                  size="Humidity",tiles="OSM",hover_cols=["Hotel Name","Country"])

# Display the map
hotel_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)